In [2]:
!pip install transformers torch nltk rouge_score

In [11]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import fastText
from contextlib import contextmanager
import time
import json
import wandb
import torch.nn.functional as F
import torch
from collections import deque
import numpy as np

In [12]:
from power_monitoring.monitor import HWMonitor
import threading
import wandb

In [13]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
# combined_outputs["gsm8k"] = [{"input_text": None, "7b": None, "tiny": None, "13b": None} for _ in range(3000)]
# len(combined_outputs["gsm8k"])

# with open('data/input_output_train', 'wb') as f:
#     pickle.dump(combined_outputs, f)

In [19]:
import pickle
import pandas as pd

with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,{'input_text': 'Gutach: Noch mehr Sicherheit f...,{'input_text': '(CNN)The Palestinian Authority...,"{'input_text': None, '7b': None, 'tiny': None,..."
1,{'input_text': 'Sie stehen keine 100 Meter von...,{'input_text': '(CNN)Never mind cats having ni...,"{'input_text': None, '7b': None, 'tiny': None,..."
2,{'input_text': 'Zwei Anlagen so nah beieinande...,{'input_text': '(CNN)If you've been following ...,"{'input_text': None, '7b': None, 'tiny': None,..."
3,{'input_text': 'Diese Frage hat Gutachs Bürger...,{'input_text': '(CNN)Five Americans who were m...,"{'input_text': None, '7b': None, 'tiny': None,..."
4,"{'input_text': '""Die Rathausampel ist damals i...",{'input_text': '(CNN)A Duke student has admitt...,"{'input_text': None, '7b': None, 'tiny': None,..."


In [16]:
df_combined_outputs.size

9000

In [17]:
df_combined_outputs["cnn_dailymail"][3]

{'input_text': "(CNN)Five Americans who were monitored for three weeks at an Omaha, Nebraska, hospital after being exposed to Ebola in West Africa have been released, a Nebraska Medicine spokesman said in an email Wednesday. One of the five had a heart-related issue on Saturday and has been discharged but hasn't left the area, Taylor Wilson wrote. The others have already gone home. They were exposed to Ebola in Sierra Leone in March, but none developed the deadly virus. They are clinicians for Partners in Health, a Boston-based aid group. They all had contact with a colleague who was diagnosed with the disease and is being treated at the National Institutes of Health in Bethesda, Maryland. As of Monday, that health care worker is in fair condition. The Centers for Disease Control and Prevention in Atlanta has said the last of 17 patients who were being monitored are expected to be released by Thursday. More than 10,000 people have died in a West African epidemic of Ebola that dates to 

In [9]:
def algorithm(T, V, h_tilde, c):
    l_predictor, s_predictor = None, None
    Q = 0.0 
    results = []

    l_loss_all = []
    s_loss_all = []
    acc_all = []
    energy_all = []

    for t in range(1, T+1):
        print(f"Index: {t}")
        t_i = df_combined_outputs["wmt14"][t]["input_text"]
        t_e = df_combined_outputs["wmt14"][t]["label"]

        p_t = min(1, c / math.sqrt(t))
        X_t = Bernoulli(p_t)

        l_loss, s_loss = None, None
        l_predicted_acc, s_predicted_acc = None, None

        g_large = 527.870
        g_small = 44.639

        # CNN Dailymail: [750.2854566666667, 142.08939366666667]

        if X_t == 1:
            t_c = getResults(t, t_i, t_e)
            l_predictor, s_predictor, l_loss, s_loss = sgdStep(l_predictor, s_predictor, t_c)
            l_predicted_acc, s_predicted_acc = predict(t_i, l_predictor, s_predictor)
            l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)
        else:
            l_predicted_acc, s_predicted_acc = predict(t_i, l_predictor, s_predictor)

        t_c = getResults(t, t_i, t_e)
        cost_large = V * g_large + Q * (h_tilde - l_predicted_acc)
        cost_small = V * g_small + Q * (h_tilde - s_predicted_acc)

        print('Q=', Q)
        if cost_large < cost_small:
            x_t = 'large'
            selected_acc = l_predicted_acc
            selected_energy = g_large
            print('selected large')
        else:
            x_t = 'small'
            selected_acc = s_predicted_acc
            selected_energy = g_small
            print('selected small')

        output, t_acc = querySelectedModel(t, t_e, x_t)

        if l_loss is not None:
            l_loss_all.append(l_loss)
        if s_loss is not None:
            s_loss_all.append(s_loss)
        acc_all.append(t_acc)
        energy_all.append(selected_energy)

        avg_l_loss = np.mean(l_loss_all)
        avg_s_loss = np.mean(s_loss_all)
        avg_acc = np.mean(acc_all)
        avg_energy = np.mean(energy_all)

        Q = max(0.0, Q + h_tilde - t_acc)
        log(results, t, t_i, output, avg_acc, avg_energy, x_t, avg_l_loss, avg_s_loss)

    return results

In [10]:
def log(results, t, t_i, output, t_acc, t_energy, x_t, l_loss, s_loss):
    results.append({
        'iteration': t,
        'input': t_i,
        'best_output': output,
        'chosen_models_avg_accuracy': t_acc,
        'chosen_models_avg_energy': t_energy,
        'chosen_model': x_t,
        'l_loss': l_loss,
        's_loss': s_loss
    })
    log_data = {
        'iteration': t,
        'chosen_model_accuracy': t_acc,
        'chosen_model_energy': t_energy
    }
        
    if l_loss is not None: log_data['l_loss'] = l_loss
    if s_loss is not None: log_data['s_loss'] = s_loss
        
    wandb.log(log_data)

In [11]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [12]:
def querySelectedModel(t, t_e, x_t):
    if x_t == "large":
        output = df_combined_outputs["wmt14"][t]["13b"]
    else:
        output =  df_combined_outputs["wmt14"][t]["tiny"]
        
    return output, calculate_bleu(output, t_e)

In [13]:
# def queryBest(t, t_i, t_e, l_predictor, s_predictor):
#     l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
#     if l_acc > s_acc: # use larger model, e.g. 7b llama
#         output = df_combined_outputs["wmt14"][t]["13b"]
#     else: # use smaller model for all other cases, e.g. 3b llama
#         output =  df_combined_outputs["wmt14"][t]["tiny"]

#     return output, calculate_bleu(output, t_e)
#     # return output, calculate_rouge(output, t_e)

In [14]:
def moving_average(data, window_size):
    data_list = list(data)  
    if len(data_list) == 0:
        return 0.0 
    if len(data_list) < window_size:
        return sum(data_list) / len(data_list)
    return sum(data_list[-window_size:]) / window_size

In [15]:
def compute_mse_loss(predicted_accuracy, true_accuracy):
    predicted_tensor = torch.tensor([predicted_accuracy], dtype=torch.float32)
    true_tensor = torch.tensor([true_accuracy], dtype=torch.float32)

    loss = F.mse_loss(predicted_tensor, true_tensor)
    
    return loss.item()

In [16]:
def sgdStep(large_model_predictor, small_model_predictor, t_c):
    predicted_large_label, predicted_small_label = 0, 0
    if large_model_predictor and small_model_predictor != None: 
        predicted_large_label = large_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
        predicted_small_label = small_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
    
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    large_model_loss = compute_mse_loss(float(predicted_large_label), t_c['large_model_accuracy'])
    small_model_loss = compute_mse_loss(float(predicted_small_label), t_c['small_model_accuracy'])

    # print(f"Large Stats(predicted, actual): {predicted_large_label}, {t_c['large_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")
    # print(f"Small Stats(predicted, actual): {predicted_small_label}, {t_c['small_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")

    return large_model_predictor, small_model_predictor, large_model_loss, small_model_loss

In [17]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [18]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["wmt14"][t]["tiny"]
    l_output = df_combined_outputs["wmt14"][t]["13b"]
    
    # WMT14
    s_acc = calculate_bleu(s_output, t_e)
    l_acc = calculate_bleu(l_output, t_e)

    # CNN Dailymail
    # s_acc = calculate_rouge(s_output, t_e)['rouge1']
    # l_acc = calculate_rouge(l_output, t_e)['rouge1']

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [19]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [20]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [21]:
def run_and_record(T, V, h_tilde, c, run_name):
    wandb.init(project="classifier", name=run_name)
    stop_event = threading.Event()
    hw_monitor = HWMonitor(monitoring_freq=1.0, stop_event=threading.Event())
    hw_monitor.start()
    
    results = algorithm(T, V, h_tilde, c)

    stop_event.set() 
    hw_monitor.join()
    wandb.finish()

    return results

In [ ]:
h_tilde_config = .55

In [ ]:
V_values = [0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]  

for V in V_values:
    run_name = f'fasttext_wmt14_results_V{V}_h{h_tilde_config}'  
    results = run_and_record(2999, V, h_tilde_config, 5, run_name)
    filename = f'experiments/fasttext/{run_name}.json'
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ryanzhangofficial (ryzhangofficial). Use `wandb login --relogin` to force relogin


Index: 1


Read 

Checkpoint
Q= 0.0
selected small
Index: 2
Checkpoint
Q= 0.04004343684405276
selected small
Index: 3
Checkpoint
Q= 0.15074736887865992
selected small
Index: 4
Checkpoint
Q= 0.4491559292286271
selected small
Index: 5
Q= 0.44885401503573696
selected small
Index: 6
Checkpoint
Q= 0.29885401503573694
selected small
Index: 7
Q= 0.4710762372579592
selected small
Index: 8
Checkpoint
Q= 0.2622027216430749
selected small
Index: 9
Checkpoint
Q= 0.3186001322981996
selected large
Index: 10
Q= 0.0
selected small
Index: 11
Q= 0.14641360910121198
selected large
Index: 12
Checkpoint
Q= 0.06049611512406705
selected small
Index: 13
Q= 0.059896751543410676
selected large
Index: 14
Q= 0.0
selected small
Index: 15
Checkpoint
Q= 0.09405173045218812
selected small
Index: 16
Checkpoint
Q= 0.31586634421548054
selected small
Index: 17
Q= 0.24465014261459817
selected small
Index: 18
Q= 0.3387668524293502
selected small
Index: 19
Q= 0.07920019976176584
selected large
Index: 20
Checkpoint
Q= 0.0
selected small
Index

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [ ]:
V_values = [0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]  

for V in V_values:
    run_name = f'fasttext_wmt14_results_V{V}_h{h_tilde_config}'  
    results = run_and_record(2999, V, h_tilde_config, 5, run_name)
    filename = f'experiments/fasttext/{run_name}.json'
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)

In [27]:
total = 0
num_results = len(data)

for result in data:
    total += result['chosen_model_accuracy']

total / num_results

0.18898695987501102

In [16]:
with open('data/cnndailymail_results.json', 'r') as f:
    data = json.load(f)

In [17]:
rouge1_total = 0
rouge2_total = 0
rougeL_total = 0

num_results = len(data)

for result in data:
    rouge1_total += result['chosen_model_accuracy']['rouge1']
    rouge2_total += result['chosen_model_accuracy']['rouge2']
    rougeL_total += result['chosen_model_accuracy']['rougeL']

rouge1_avg = rouge1_total / num_results
rouge2_avg = rouge2_total / num_results
rougeL_avg = rougeL_total / num_results

print(f"Average ROUGE-1: {rouge1_avg}")
print(f"Average ROUGE-2: {rouge2_avg}")
print(f"Average ROUGE-L: {rougeL_avg}")

Average ROUGE-1: 0.2937870737089562
Average ROUGE-2: 0.10789698165405123
Average ROUGE-L: 0.19841589354653624


In [18]:
import fasttext
import wandb
import pickle
import pandas as pd

wandb.init(project="fasttext_training", name="FastText_First_1000_Samples")
with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)

df_samples = pd.concat([
    pd.DataFrame(df_combined_outputs['wmt14'][:1000]),
    pd.DataFrame(df_combined_outputs['cnn_dailymail'][:1000]),
    pd.DataFrame(df_combined_outputs['gsm8k'][:1000])
])

train_data = [
    f"__label__{row['label']} {row['input_text']}" for idx, row in df_samples.iterrows()
]

train_file = 'fasttext_train_1000_data.txt'
with open(train_file, 'w') as f:
    for line in train_data:
        f.write(line + '\n')

class LossLogger:
    def __init__(self):
        self.losses = []

    def log(self, epoch, loss):
        self.losses.append(loss)
        wandb.log({"epoch": epoch, "loss": loss})

loss_logger = LossLogger()

model = fasttext.train_supervised(
    input=train_file,
    epoch=5,
    lr=0.1,
    wordNgrams=2,
    verbose=1,
    callback=loss_logger.log  
)

model.save_model("fasttext_model.bin")

wandb.finish()

print("Training complete. Model saved.")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2646403/1328822952.py", line 1, in <module>
    import fasttext
ModuleNotFoundError: No module named 'fasttext'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/ubuntu/.local/lib/python3.8/site-packages/IPython/c